**CNN with Adam Optimizer and a LR Scheduler**
Image Transformations Used:

Normalization
Params:

Epochs - 30
Learning Rate - 0.001

In [ ]:
import os
import shutil
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_recall_fscore_support
)

from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision.datasets import GTSRB
from torchvision import models

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split


In [ ]:
!ls /content/

drive  sample_data


In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
root_dir = '/root/GTSRB'

os.makedirs(root_dir, exist_ok=True)

# Applying the custom transform in the dataset loader
root_dir = '/root/GTSRB'
os.makedirs(root_dir, exist_ok=True)


In [ ]:
# Load datasets
gtsrb_train = GTSRB(root=root_dir, split='train', transform=transform, download=True)

# Load the test dataset
gtsrb_test = GTSRB(root=root_dir, split='test', transform=transform, download=True)

# Calculate the size of each split (half of the test dataset)
test_size = len(gtsrb_test)
validation_size = test_size // 2
test_size = test_size - validation_size  # The other half for testing

# Split the test data into validation and test
validation_dataset, test_dataset = random_split(gtsrb_test, [validation_size, test_size])




100%|██████████| 187M/187M [00:07<00:00, 23.9MB/s]


Extracting /root/GTSRB/gtsrb/GTSRB-Training_fixed.zip to /root/GTSRB/gtsrb


100%|██████████| 89.0M/89.0M [00:04<00:00, 18.4MB/s]


Extracting /root/GTSRB/gtsrb/GTSRB_Final_Test_Images.zip to /root/GTSRB/gtsrb


100%|██████████| 99.6k/99.6k [00:00<00:00, 220kB/s]


Extracting /root/GTSRB/gtsrb/GTSRB_Final_Test_GT.zip to /root/GTSRB/gtsrb


In [ ]:
print(f'Number of training samples: {len(gtsrb_train)}')
print(f'Number of total test samples: {len(gtsrb_test)}')
print(f'Number of valid samples: {len(validation_dataset)}')
print(f'Number of test samples: {len(test_dataset)}')

# Determine the number of classes by looking at unique labels in the dataset
train_labels = [label for _, label in gtsrb_train]
test_labels = [label for _, label in test_dataset]
vlaidation_labels = [label for _, label in validation_dataset]

num_classes = len(set(train_labels))
print(f'Number of classes: {num_classes}')

Number of training samples: 26640
Number of total test samples: 12630
Number of valid samples: 6315
Number of test samples: 6315


In [ ]:
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader = DataLoader(gtsrb_train, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# plot for RGB
def plot_random_images(dataset, num_images=5):
    indices = random.sample(range(len(dataset)), num_images)
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

    for i, idx in enumerate(indices):
        image, label = dataset[idx]
        if image.shape[0] == 3:
            image = np.transpose(image, (1, 2, 0))
        axes[i].imshow(image)
        axes[i].set_title(f'Label: {label}')
        axes[i].axis('off')

    plt.show()


# Plot 5 random images from the training dataset
plot_random_images(gtsrb_train, num_images=5)


In [ ]:
# Load pretrained MobileNetV2 model (for RGB images, 3 channels)
model = models.mobilenet_v2(pretrained=True)

# Modify the final classifier layer to match the number of classes in GTSRB
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100 * correct / total
    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.to(device).float()
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()

            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_accuracies.append(val_accuracy)
    val_losses.append(val_running_loss / len(validation_loader))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_running_loss/len(validation_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Accuracy: {val_accuracy:.2f}%")

# Save model weights
# torch.save(model.state_dict(), "/content/drive/MyDrive/mobilenetv2_gtsrb_Neha_Kim_Jonathan.pth")

# Plotting training and validation loss
plt.figure(figsize=(15, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()

# Plotting training and validation accuracy
plt.figure(figsize=(15, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()

# Evaluation on test data
model.eval()
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device).float(), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Compute precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


In [ ]:
# Load pretrained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device).float()
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100 * correct / total
    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.to(device).float()
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()

            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_accuracies.append(val_accuracy)
    val_losses.append(val_running_loss / len(validation_loader))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_running_loss/len(validation_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Accuracy: {val_accuracy:.2f}%")

# Save model weights
# torch.save(model.state_dict(), "/content/drive/MyDrive/resnet_gtsrb_Neha_Kim_Jonathan.pth")

# Plotting training and validation loss
plt.figure(figsize=(15, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()

# Plotting training and validation accuracy
plt.figure(figsize=(15, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()

# Evaluation on test data
model.eval()
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device).float(), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Compute precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


In [ ]:
# EOF